# LDA(Latent Dirichlet Allocation): 잠재 디리클레 할당

https://wikidocs.net/40710 참고

In [1]:
import pymysql

DB_HOST = 'localhost'
DB_USER = 'pyuser'
DB_PASSWD = 'pyuser'
DB_NAME = 'pyuser'

conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                   db=DB_NAME, charset='utf8')
curs = conn.cursor()
sql = """
    select * 
    from fmkorea
;

"""
curs.execute(sql)
table_data = curs.fetchall() 
conn.close()

In [2]:
import pandas as pd

df = pd.DataFrame(table_data,columns=['no','url','Title','Writer','ContDate', 'content','reply_cnt','reply_content'])
# df = pd.DataFrame(table_data,columns=['Contents'])

# df = pd.DataFrame(table_data)
# df.columns=['no','url','Date','Title','subtitle','Contents']

# 데이터 유무 확인. 분석건수 확인. 
df.head()

df.info()  # 전체 데이터 건수 확인

df.shape   # 전체 데이터 건수 확인

# df [ 'Contents'   ]  <= Series타입,   df[ ['Contents'] ]  <== dataframe타입
sentences = df[ ['content'] ]   

# print로 type(text) 의 결과는 Dataframe타입이어야 함
print(type(sentences))
# <class 'pandas.core.frame.DataFrame'> 이 아니면 오류임

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   no             309 non-null    int64 
 1   url            309 non-null    object
 2   Title          309 non-null    object
 3   Writer         309 non-null    object
 4   ContDate       309 non-null    object
 5   content        309 non-null    object
 6   reply_cnt      309 non-null    object
 7   reply_content  309 non-null    object
dtypes: int64(1), object(7)
memory usage: 19.4+ KB
<class 'pandas.core.frame.DataFrame'>


라이브러리 추가 및 클래스 선언

In [3]:
# konlpy 설치 명령어
# !pip install konlpy

In [4]:
from konlpy.tag import Okt
okt=Okt()

명사추출: nouns 명사추출. phrases 어절 추출

In [5]:
tokenized_doc = sentences.apply(lambda row: okt.nouns(row['content']), axis=1)


단어 길이가 1이하인 단어들은 제거 (길이가 짧은 단어 ex. 은/는/이/가/을/를 등 제거)


In [6]:
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if len(word) > 1])


분석에서 제거할 불필요한 단어들 기재

In [7]:
from nltk.corpus import stopwords

stop = ['연합뉴스','무단']
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if word not in (stop)])


In [8]:
# 저장된 단어들 확인하기 
print(tokenized_doc)

0      [손해, 안보, 피붕, 이임, 종종, 채팅, 때문, 고소, 우선, 채팅, 절대, 모...
1      [채팅, 게임, 시작, 생략, 전체, 대화, 상황, 상황, 세레모니, 상황, 부부,...
2      [대충, 음성, 어플, 대부분, 녹취본, 가지, 저걸, 처벌, 처벌, 실명, 가지,...
3                       [매음, 고소, 진짜, 건수, 이상, 애초, 법인, 지도]
4                                                     []
                             ...                        
304                                                   []
305    [토요일, 새벽, 관계, 증거, 자료, 내용, 국민신문고, 인근, 경찰서, 담당자,...
306                                                   []
307                                     [매음, 준비, 시작, 시작]
308    [친구, 시작, 자마자, 선제골, 자기, 우리, 구단, 가치, 일임, 정도, 점점,...
Length: 309, dtype: object


TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 
여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치

https://wikidocs.net/31698 참고

TF-IDF의 TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용함.
이를 사용하기 위해 다시 토큰화 작업을 역으로 취소하는 역토큰화(Detokenization)작업을 수행

# 사이킷런의 잠재 디리클레 할당(LDA) 실습

https://wikidocs.net/40710

In [9]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []

for i in range(len(sentences)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

sentences['content'] = detokenized_doc

print(sentences['content'][:5])

0    손해 안보 피붕 이임 종종 채팅 때문 고소 우선 채팅 절대 모욕 명예훼손 피붕이들 ...
1    채팅 게임 시작 생략 전체 대화 상황 상황 세레모니 상황 부부 스킵 상황 상황 저희...
2                대충 음성 어플 대부분 녹취본 가지 저걸 처벌 처벌 실명 가지 레벨
3                              매음 고소 진짜 건수 이상 애초 법인 지도
4                                                     
Name: content, dtype: object


C:\Users\ksj31\AppData\Local\Temp\ipykernel_8828\3327310773.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentences['content'] = detokenized_doc


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer(max_features= 1000)    # 상위 1,000개의 단어를 보존 

# TfidfVectorizer 할 X_train 데이터
X = vectorizer.fit_transform(sentences['content'])

from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델훈련
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_model.fit_transform(X)

# vocabulary를 활용하여 각 문장이 갖고 있는 토큰의 count를 기반으로 문장을 vectorization 처리
terms = vectorizer.get_feature_names() 

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

#  토픽별 단어 분포
get_topics(lda_model.components_,terms)



Topic 1: [('부류', 1.34), ('사람', 1.09), ('지원', 1.04), ('브라우저', 1.02), ('등등', 0.99), ('특정', 0.97), ('존나', 0.97), ('부정', 0.96), ('인터넷', 0.96), ('고소', 0.94)]
Topic 2: [('매음', 12.89), ('고소', 5.56), ('드립', 4.56), ('진짜', 3.21), ('새끼', 3.09), ('친구', 3.07), ('생각', 2.93), ('처벌', 2.63), ('그냥', 2.38), ('상대', 2.33)]
Topic 3: [('기분', 2.2), ('존나', 1.74), ('쪽지', 1.57), ('손해', 1.54), ('잉여', 1.37), ('선물', 1.35), ('부터', 1.32), ('자동', 1.31), ('발송', 1.3), ('신고', 1.24)]
Topic 4: [('지랄', 1.54), ('경찰서', 1.53), ('매음', 1.35), ('패드', 1.3), ('부모님', 1.24), ('캡쳐', 1.16), ('채팅', 1.14), ('고소장', 1.08), ('수도', 1.03), ('요즘', 1.01)]
Topic 5: [('여기', 1.77), ('디씨', 1.22), ('실명', 1.21), ('니야', 1.2), ('인증', 1.2), ('인신공격', 1.19), ('악플', 1.09), ('타임', 1.01), ('게뭐', 0.99), ('매음', 0.94)]
Topic 6: [('헌터', 1.4), ('합의금', 1.38), ('신고', 1.29), ('매음', 1.27), ('바로', 1.25), ('유도', 1.23), ('병신', 1.2), ('경찰서', 1.19), ('드립', 1.17), ('방문', 1.13)]
Topic 7: [('드립', 1.09), ('존버', 1.06), ('나라', 1.03), ('사유', 0.93), ('매너', 0.86), ('나머지', 0.85), (

C:\Users\ksj31\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# gensim을 통해서 LDA를 수행하고, 시각화

https://wikidocs.net/30708

In [11]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

In [17]:
import gensim

NUM_TOPICS = 5 # n개의 토픽, k=n

# passes는 알고리즘의 동작 횟수
# 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 설정함
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)   # TOPIC별 단어를 4개만 보여주기, 숫자를 늘리면 더 많은 단어들이 보여짐
for topic in topics:
    print(topic)


(0, '0.046*"매음" + 0.018*"드립" + 0.013*"상대" + 0.011*"진짜"')
(1, '0.059*"매음" + 0.017*"게임" + 0.016*"존나" + 0.013*"사람"')
(2, '0.025*"고소" + 0.016*"매음" + 0.012*"접수" + 0.012*"경찰서"')
(3, '0.031*"매음" + 0.021*"고소" + 0.017*"새끼" + 0.015*"드립"')
(4, '0.039*"매음" + 0.033*"드립" + 0.016*"게임" + 0.014*"친구"')


각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줌
또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 N개의 토픽은 0부터 N-1까지의 번호가 할당되어져 있음

위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미함
예를 들어 0번째 문서의 토픽 비율에서 (0, '0.023*"경제..."'')은 0번 토픽에서 경제가 2.3%의 분포도를 가지는 것을 의미함

In [15]:
# !pip install pyLDAvis

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136882 sha256=c9495ee42e89220daa6015d82c9a7432002c82f65837c0fd59e8e68770c565f0
  Stored in directory: c:\users\ksj31\appdata\local\pip\cache\wheels\57\a4\86\d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=e723f154ab29d3fa27d8842647102b689ba430e61a54f8c9b5b6771dd223dd38
  Stored in directory: c:\users\ksj31\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully

In [18]:

import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

# https://lovit.github.io/nlp/2018/09/27/pyldavis_lda/
# 시각화는 1- 부터 시작함
pyLDAvis.display(vis)

C:\Users\ksj31\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


λ라는 값은 [0, 1] 사이에서 조절가능함
λ를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미이며,
λ를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미임